In [1]:
import numpy as np
import lda
import lda.datasets
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()

print(vocab)

print(type(X))
X.shape
(395, 4258)



model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(X)
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

('church', 'pope', 'years', 'people', 'mother', 'last', 'told', 'first', 'world', 'year', 'president', 'teresa', 'charles', 'catholic', 'during', 'life', 'u.s', 'city', 'public', 'time', 'since', 'family', 'king', 'former', 'british', 'harriman', 'against', 'country', 'vatican', 'made', 'three', 'hospital', 'minister', 'home', 'died', 'tuesday', 'government', "n't", 'million', 'prince', 'john', 'very', 'war', 'say', 'order', 'day', 'political', 'leader', 'heart', 'roman', 'yeltsin', 'later', 'france', 'party', 'monday', 'death', 'clinton', 'sunday', 'elvis', 'state', 'diana', 'royal', 'wednesday', 'churchill', 'official', 'service', 'including', 'film', 'while', 'week', 'left', 'work', 'paris', 'around', 'american', 'saying', 'take', 'doctors', 'group', 'son', 'queen', 'house', 'police', 'national', 'under', 'surgery', 'next', 'peace', 'paul', 'long', 'ceremony', 'month', 'thursday', 'french', 'head', 'marriage', 'operation', 'visit', 'show', 'germany', 'russian', 'days', 'love', 'news

In [8]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import operator
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

path_titles_meta = "acl-metadata.txt"




f = open(path_titles_meta, "r", encoding="Latin-1")

# compile titles into a list
titles = []
titlesDict = {}
# list for tokenized documents in loop
texts = []

#Extracting the titles from our meta-document
for line in f:
    if line.startswith('title'):
        tmp = line.split(' = ',2)
        title = tmp[1][1:len(tmp[1])-2] #last part: subtracts /n}; 
        titles.append(title)
        titlesDict[title] = None



# loop through title list    
for i in titles:
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    #show tokenized words
    #print(tokens)
    
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
   
    
    # stem token "-e are lost??" 
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    #print(stemmed_tokens)
    
    # add tokens to list
    texts.append(stopped_tokens)
    
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)



# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print(max([v for s in corpus for v, _ in s]))

# generate LDA model
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)

#print(ldamodel.print_topics(num_topics=5, num_words=5))


print(len(titlesDict.keys()))







    

    

10434
19272


In [12]:
from scipy.sparse import csr_matrix
dictionary_len = max([v for s in corpus for v, _ in s]) + 1
document_len = len(corpus)
converted_input = csr_matrix((document_len, dictionary_len), dtype=np.int16).toarray()
print(converted_input.shape)
tup_dict = corpus

for i in range(len(tup_dict)):
    for (word_idx, count) in tup_dict[i]:
        converted_input [i, word_idx] = count
        
corpus = converted_input

(19424, 10435)


In [3]:
vocab = []
for i in range(len(dictionary.keys())):
    vocab.append(dictionary.get(i))

In [16]:
import lda
import lda.datasets


model = lda.LDA(n_topics=10, n_iter=500, random_state=1)
model.fit(corpus)
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    

Topic 0: translation machine statistical based phrase models model
Topic 1: language natural system processing understanding text muc
Topic 2: task corpus system annotation lexical english resources
Topic 3: discourse resolution semantics expressions structure relations semantic
Topic 4: information text extraction automatic summarization using document
Topic 5: speech word segmentation chinese recognition based tagging
Topic 6: semantic learning using entity based named extraction
Topic 7: dialogue system spoken systems question answering speech
Topic 8: parsing grammar dependency grammars based tree parser
Topic 9: word using semantic sense disambiguation corpora based


In [17]:
cls = model.transform(corpus)

In [25]:
print(str(titles[0]) + " " + str(sum(cls[0])))

On Dual Decomposition and Linear Programming Relaxations for Natural Language Processing 1.0


In [21]:
print(cls)

[[  2.38101563e-03   5.08116203e-01   1.88984235e-04 ...,   1.10990872e-03
    4.83456293e-01   3.71172180e-05]
 [  3.12578534e-03   1.71827469e-05   1.74323423e-05 ...,   2.02850931e-04
    1.74136511e-01   6.69978849e-04]
 [  1.06211220e-02   9.59833830e-05   9.73807811e-05 ...,   9.02747207e-05
    4.41382039e-01   3.48628315e-02]
 ..., 
 [  3.85420805e-01   8.22656858e-04   3.60968487e-03 ...,   9.16467163e-03
    2.65831058e-01   5.16231324e-03]
 [  8.34445220e-04   9.07619963e-02   5.27530529e-03 ...,   5.11023519e-02
    5.70016574e-01   1.54159639e-04]
 [  4.29919164e-01   9.02211774e-05   5.83253155e-04 ...,   1.57704693e-03
    4.70973438e-01   8.00296463e-02]]


In [28]:
path_titles_meta_lda = "acl-metadata-lda.txt"


f = open(path_titles_meta_lda, "w", encoding="Latin-1")

for i in range(len(titles)):
    f.write(str(titles[i]) + "\t" + str(cls[i]) + "\n")
f.close()

